In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# **** USER INPUTS *****
user_email = ""
user_password = ""
url = "https://susyandbae.com/collections/lingerie"
# url = "https://susyandbae.com/collections/moments-in-bloom"
size = "M"
quantity = int(2)

# Initialize the WebDriver
driver = webdriver.Chrome()

# List of links
links = []

def next_page():
    
    # Find and click next page button
    try: 
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'Pagination__Nav')))
        next_page_button = driver.find_element(By.XPATH, '//*[@title="Next page"]')
        next_page_button.click()
        print('Scraping products in next page')
    except: 
        pass

def get_susyandbae_links():

    # Open the URL
    driver.get(url)
    time.sleep(2)
    
    # Get last page number
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'Pagination__Nav')))
    last_page = driver.find_element(By.XPATH, "//div[@class='Pagination__Nav']")
    last_page = int(last_page.text[-1])

    for i in range(last_page):
        # Wait until the listings are loaded
        time.sleep(2)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "ProductItem")))

        # Get all product links and save in list
        products = driver.find_elements(By.CLASS_NAME, "ProductItem")
        for product in products:
            links.append(product.find_element(By.TAG_NAME, "a").get_attribute("href"))

        next_page()
    print('Count of links: ', len(links))

def purchase_susyandbae():
    try:
        for link in links:
            # Open product link in a new tab
            print('Loading webpage: ', link)
            driver.execute_script(f"window.open('{link}', '_blank');")
            driver.switch_to.window(driver.window_handles[-1])

            # Wait for the size dropdown to be available
            size_select = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "ProductForm__Variants"))
            )
            
            # Select size 
            for option in size_select.find_elements(By.CLASS_NAME, 'SizeSwatch'):
                if option.text.strip() == size:
                    option.click()
                    time.sleep(0.1)
                    print(f'Selected Size "{size}" for {link}')
                    break

            # Select quantity
            try:
                inputQuantity = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@aria-label="Quantity"]')))
                inputQuantity.send_keys(Keys.BACKSPACE)
                inputQuantity.send_keys(quantity)
                print(f'Quantity of 2 entered for {link}')
            except: 
                print(f"Quantity 1: Unable to change quantity for {link}")

            try: 
                # Check if sold out
                soldOutCheck = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "ProductForm__AddToCart")))
                
                if soldOutCheck.text.lower() == "sold out":
                    print(f'Sold out: {link} in size {size} is sold out. Trying next item.')
                    time.sleep(0.15)
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    continue

                else:
                    add_to_cart_button = driver.find_element(By.CLASS_NAME, "ProductForm__AddToCart")
                    add_to_cart_button.click()
                    time.sleep(0.2)
                    print(f'Added to cart: {link}. Quantity: {quantity}. Size: {size}.')

                    # Close the current tab and switch back to the product listing tab
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

            except: 
                pass

    finally:
        # Go to cart
        go_to_cart_button = driver.find_element(By.CLASS_NAME, "Header__CartCount")
        go_to_cart_button.click()

        # # login to account
        # account_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "Heading")))
        # account_button.click
        # email = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@aria-label="Email"]')))
        # email.send_keys(user_emai)
        # password = find_element((By.XPATH, '//*[@aria-label="Password"]'))
        # password.send_keys(user_password)
        # login_button = find_element(By.CLASS_NAME, "Form__Submit")
        # login_button.click()

        # # Checkout
        # checkout_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "Cart__Checkout")))
        # checkout_button.click()

get_susyandbae_links()
purchase_susyandbae()